In [114]:
!pip install sqlalchemy

In [115]:
import sqlalchemy as db

In [116]:
dir(db)

['ARRAY',
 'AdaptedConnection',
 'Alias',
 'AliasedReturnsRows',
 'Any',
 'AssertionPool',
 'AsyncAdaptedQueuePool',
 'BIGINT',
 'BINARY',
 'BLANK_SCHEMA',
 'BLOB',
 'BOOLEAN',
 'BaseDDLElement',
 'BaseRow',
 'BigInteger',
 'BinaryExpression',
 'BindParameter',
 'BindTyping',
 'Boolean',
 'BooleanClauseList',
 'CHAR',
 'CLOB',
 'CTE',
 'CacheKey',
 'Case',
 'Cast',
 'CheckConstraint',
 'ChunkedIteratorResult',
 'ClauseElement',
 'ClauseList',
 'CollectionAggregate',
 'Column',
 'ColumnClause',
 'ColumnCollection',
 'ColumnDefault',
 'ColumnElement',
 'ColumnExpressionArgument',
 'ColumnOperators',
 'Compiled',
 'CompoundSelect',
 'Computed',
 'Connection',
 'Constraint',
 'CreateEnginePlugin',
 'CursorResult',
 'DATE',
 'DATETIME',
 'DDL',
 'DDLElement',
 'DECIMAL',
 'DOUBLE',
 'DOUBLE_PRECISION',
 'Date',
 'DateTime',
 'DefaultClause',
 'Delete',
 'Dialect',
 'Double',
 'Engine',
 'Enum',
 'ExceptionContext',
 'Executable',
 'ExecutableDDLElement',
 'ExecutionContext',
 'Exists',
 'Ex

In [228]:
def inspector() -> None:
  inspector = db.inspect(engine)
  tables = inspector.get_table_names()
  print(tables)
  print("Для просмотра таблицы, введите view_table(table_name).")
  print("Например для 'Classroom' будет view_table(classroom).")

def view_table(metadates: db.Table) -> None:
  select_all_query = db.select(metadates)
  select_result = conn.execute(select_all_query)
  result = select_result.all()
  for row in result:
    print(row)

def view_mod_table(metadates: db.Table) -> None:
  select_result = conn.execute(metadates)
  result = select_result.all()
  for row in result:
    print(row)

In [118]:
engine = db.create_engine('sqlite:///hometask.db')

In [119]:
conn = engine.connect()

In [120]:
metadata = db.MetaData()

In [121]:
student = db.Table(
    'Student', metadata,
    db.Column('id', db.Integer, primary_key=True),
    db.Column('student_name', db.Text,),
)

In [122]:
student_in_class = db.Table(
    'Student_in_class', metadata,
    db.Column('id', db.Integer, primary_key=True),
    db.Column('class_id', db.Integer,),
    db.Column('student', db.Integer,),
)

In [123]:
class_name = db.Table(
    'Class', metadata,
    db.Column('id', db.Integer, primary_key=True),
    db.Column('name', db.Text,),
)

In [124]:
classroom = db.Table(
    'Classroom', metadata,
    db.Column('id', db.Integer, primary_key=True),
    db.Column('class_id', db.Integer,),
    db.Column('classroom', db.Integer,),
)

In [125]:
metadata.create_all(engine)

In [126]:
insertion_student = student.insert().values([
    {'student_name': 'Petr Petrov'},
    {'student_name': 'Maria Ivanova'},
    {'student_name': 'Ivan Sinichkin'},
    {'student_name': 'Anastasia Gromova'},
    {'student_name': 'Nikolai Pechkin'},
    {'student_name': 'Polina Gagarina'},
])

In [127]:
conn.execute(insertion_student)

In [130]:
insertion_student_in_class = student_in_class.insert().values([
    {'class_id': 1, 'student': 1},
    {'class_id': 1, 'student': 2},
    {'class_id': 2, 'student': 3},
    {'class_id': 2, 'student': 4},
    {'class_id': 3, 'student': 5},
    {'class_id': 3, 'student': 6},
])

In [131]:
conn.execute(insertion_student_in_class)

In [132]:
insertion_class_name = class_name.insert().values([
    {'name': '10 A'},
    {'name': '8 B'},
    {'name': '6 A'},
])

In [133]:
conn.execute(insertion_class_name)

In [134]:
insertion_classroom = classroom.insert().values([
    {'class_id': 1, 'classroom': 101},
    {'class_id': 2, 'classroom': 202},
    {'class_id': 3, 'classroom': 303},
])

In [135]:
conn.execute(insertion_classroom)

In [136]:
inspector()

['Class', 'Classroom', 'Student', 'Student_in_class']
Для просмотра таблицы, введите view_table(table_name).
Например для 'Classroom' будет view_table(classroom).


In [137]:
view_table(student)

(1, 'Petr Petrov')
(2, 'Maria Ivanova')
(3, 'Ivan Sinichkin')
(4, 'Anastasia Gromova')
(5, 'Nikolai Pechkin')
(6, 'Polina Gagarina')


In [155]:
left_join_query = (                           # OUTER LEFT JOIN (OUTER JOIN)
    student
    .join(student_in_class, student.c.id == student_in_class.c.student, isouter=True)
    .join(class_name, student_in_class.c.class_id == class_name.c.id, isouter=True)
    .join(classroom, class_name.c.id == classroom.c.class_id, isouter=True)
)
view_table(left_join_query)

(1, 'Petr Petrov', 1, 1, 1, 1, '10 A', 1, 1, 101)
(2, 'Maria Ivanova', 2, 1, 2, 1, '10 A', 1, 1, 101)
(3, 'Ivan Sinichkin', 3, 2, 3, 2, '8 B', 2, 2, 202)
(4, 'Anastasia Gromova', 4, 2, 4, 2, '8 B', 2, 2, 202)
(5, 'Nikolai Pechkin', 5, 3, 5, 3, '6 A', 3, 3, 303)
(6, 'Polina Gagarina', 6, 3, 6, 3, '6 A', 3, 3, 303)


In [200]:
inner_join_query = (                      # INNER LEFT JOIN (JOIN)
    student
    .join(student_in_class, student.c.id == student_in_class.c.student) # isouter=False (по умол.)
    .join(class_name, student_in_class.c.class_id == class_name.c.id) # isouter=False (по умол.)
    .join(classroom, class_name.c.id == classroom.c.class_id) # isouter=False (по умол.)
)
view_table(inner_join_query)

(1, 'Petr Petrov', 1, 1, 1, 1, '10 A', 1, 1, 101)
(2, 'Maria Ivanova', 2, 1, 2, 1, '10 A', 1, 1, 101)
(3, 'Ivan Sinichkin', 3, 2, 3, 2, '8 B', 2, 2, 202)
(4, 'Anastasia Gromova', 4, 2, 4, 2, '8 B', 2, 2, 202)
(5, 'Nikolai Pechkin', 5, 3, 5, 3, '6 A', 3, 3, 303)
(6, 'Polina Gagarina', 6, 3, 6, 3, '6 A', 3, 3, 303)


In [231]:
update_query = student_in_class.update().values(class_id=2).where(
    student_in_class.c.student == db.select(                               # Подзапрос == db.select()
        student.c.id).where(student.c.student_name == 'Polina Gagarina'
    )
)

conn.execute(update_query)

<ipython-input-231-129652274f15>:2: SAWarning: implicitly coercing SELECT object to scalar subquery; please use the .scalar_subquery() method to produce a scalar subquery.
  student_in_class.c.student == db.select(                               # Подзапрос == db.select()


In [232]:
update_query = student_in_class.update().values(class_id=None).where(
    student_in_class.c.student == db.select(                               # Подзапрос == db.select()
        student.c.id).where(student.c.student_name == 'Polina Gagarina'
    )
)

conn.execute(update_query)

<ipython-input-232-c39eca6c5fae>:2: SAWarning: implicitly coercing SELECT object to scalar subquery; please use the .scalar_subquery() method to produce a scalar subquery.
  student_in_class.c.student == db.select(                               # Подзапрос == db.select()


In [233]:
view_table(inner_join_query)

(1, 'Petr Petrov', 1, 1, 1, 1, '10 A', 1, 1, 101)
(2, 'Maria Ivanova', 2, 1, 2, 1, '10 A', 1, 1, 101)
(3, 'Ivan Sinichkin', 3, 2, 3, 2, '8 B', 2, 2, 202)
(4, 'Anastasia Gromova', 4, 2, 4, 2, '8 B', 2, 2, 202)
(5, 'Nikolai Pechkin', 5, 3, 5, 3, '6 A', 3, 3, 303)


In [243]:
having_303 = db.select(inner_join_query).group_by(student.c.student_name).having(classroom.c.classroom == 303)

view_mod_table(having_303)

(5, 'Nikolai Pechkin', 5, 3, 5, 3, '6 A', 3, 3, 303)


In [248]:
union_query = db.union(
    db.select(class_name.c.name.label('classes')),
    db.select(classroom.c.classroom.label('classes')),
)

view_mod_table(union_query)

(101,)
(202,)
(303,)
('10 A',)
('6 A',)
('8 B',)
